In [1]:
import sys
sys.path.append('../_amigocloud')

In [2]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

## ejecutar query

In [5]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    # variable para almacenar resultado
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

## ejecuta query segun id_query (ejecucion de queryes de un proyecto)

In [6]:
# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query, tipo_sql):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_project, query, tipo_sql)
    # retorna el numero de filas afectadas por el query
    return respuesta_post

## convertir dic en obj

In [7]:
# convierte un dict a un obj
# recibe el dict y el nombre con el que se creara el obj
def convertir_dict_obj(diccionario, name):
    return collections.namedtuple(name, diccionario.keys())(*diccionario.values())

In [8]:
id_proyecto = 34147
query_cargar_nom_prop = 6480

In [9]:
ejecutar_query_por_id(id_proyecto, query_cargar_nom_prop, 'post')

{'query': "update dataset_342756\r\nset \r\n    propiedad = cat.unidad_01 || ' / ' || cat.unidad_02\r\nfrom dataset_342758 cat where ST_INTERSECTS(\r\n    ubicacion, \r\n    cat.wkb_geometry)",
 'count': 1}

In [10]:
query = f'select * from dataset_342756 where informe_generado = false'
res = ejecutar_query_sql(id_proyecto, query, 'get')
res = res['data']

In [11]:
reg = res[0]

In [12]:
reg['fecha_registro'] = convertir_formato_fecha(reg['fecha_registro'])

In [13]:
ficha = convertir_dict_obj(reg, 'ficha')

In [14]:
ficha

ficha(fecha_registro='07/08/2024', amigo_id='64eb211e405d43afa192d8143e919e0b', id=5, responsable_tecnico='KOLIN USTAREZ', ubicacion='0101000020E6100000E4EE18C341994FC06C47CC47AD3D31C0', canhero='1530 / C.I.T.T.C.A.', propiedad='299 / C.I.T.T.C.A', zona='Saavedra', f1=False, f2=False, f3=True, f4=True, f5=False, f6=False, f7=False, f8=False, f9=False, f10=True, f11=True, f12=False, f13=False, f14=False, f15=True, f16=True, f17=False, f18=False, f19=False, f20=False, f21=True, f22=True, f23=False, f24=False, f25=False, f26=True, f27=True, f28=False, observaciones='Sin obs', informe_generado=False)

In [15]:
doc = DocxTemplate(ruta + "/templates/tpl_rpt_ficha_tec_agricola_bpl.docx")

context = {'ficha':ficha }

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
fech = ficha.fecha_registro.replace('/','-')
nom = ficha.canhero.replace(' / ','_')
file_name = fech + '_' + nom

doc.save(ruta + '/informes/_' + file_name + '.docx')